## 線形代数ライブラリの使用

In [77]:
using LinearAlgebra
using Combinatorics
using SparseArrays

## **Pauli 行列**

In [2]:
const σ⁰ = [
    1.0 0.0
    0.0 1.0
];

const σ¹ = [
    0.0 1.0
    1.0 0.0
];

const σ² = [
    0.0 -1.0im
    1.0im 0.0
];

const σ³ = [
    1.0 0.0
    0.0 -1.0
];

const σ⁺ = [
    0.0 1.0
    0.0 0.0
];

const σ⁻ = [
    0.0 0.0
    1.0 0.0
];

## **Jordan-Wigner 変換**

$$
\begin{align*}
c_k = (-\sigma^3_1) \otimes (-\sigma^3_2) \otimes \cdots \otimes (-\sigma^3_{k-1}) \otimes \sigma^-_k \otimes \sigma^0_{k+1} \otimes \cdots \otimes \sigma^0_{N} 
\end{align*}
$$
具体的な行列要素は
$$
\begin{align*}
(c_k)_{(i_1, \dots, i_N), (j_1, \dots, j_N)} = (-1)^{k-1} (-1)^{i_1 + i_2 + \cdots + i_{k-1}} \delta_{(i_1, \dots, i_{k-1}), (j_1, \dots, j_{k-1})} \delta_{(i_{k+1}, \dots, i_N), (j_{k+1}, \dots, j_N)} \delta_{i_k, 1} \delta_{j_k, 0}
\end{align*}
$$
で与えられる。ただし 添字は$i = 0, 1$を取るものとする。

In [215]:
function JordanWigner(num_sites::Integer, index::Integer)::Array{Real, 2}
    """
    Args:
        num_sites (Int): the number of lattice sites
        index (Int): lattice intex
    Returns:
        Matrix{Float64}
    """
    pauli_strings = [-σ³ for i in 1:index-1]
    append!(pauli_strings, [σ⁻]) # ! は破壊的処理(pythonで言うところの in_place = True) を意味する
    append!(pauli_strings, [σ⁰ for i in index+1:num_sites])
    return kron(pauli_strings...)
end;

function cop(num_modes::Integer, index::Integer)
    dim = 2^num_modes
    cop_mat = spzeros(dim, dim) #ゼロで初期化
    base = reverse([2^i for i in 0:num_modes-1])
    for binary_str in Iterators.product(collect(Iterators.repeated([0 1], num_modes-1))...)
        binary_list = collect(binary_str)
        insert!(binary_list, index, 1) # [i_1 ... i_k=1 ... i_N]という Array の作成
        ind_left = dot(binary_list, base) + 1 # ２進数表記から整数へ変換(1から始まることに注意)
        binary_list[index] = 0 # [i_1 ... i_k=0 ... i_N] という Array の作成
        ind_right = dot(binary_list, base) + 1 # ２進数表記から整数へ変換(1から始まることに注意)
        sgn = sum(binary_list[1:index-1]) + index - 1 
        cop_mat[ind_left, ind_right] = (-1)^sgn　# 行列要素を代入
    end

    return cop_mat
end;

function cdg(num_modes::Integer, index::Integer)
    return cop(num_modes, index)'
end;

function nop(num_modes::Integer, index::Integer)
    return cdg(num_modes, index)*cop(num_modes, index)
end;

## 粒子数セクターを取り出すための関数

In [230]:
function get_index_from_n(n::Integer, num_strings::Integer) # ２進数表記した時に各桁の和が n になる整数を返す
    seed_vector = zeros(Integer, num_strings)
    for i in 1:n
        seed_vector[i] = 1
    end
    base = reverse([2^i for i in 0:num_strings-1])

    return [dot(binary_string, base) for binary_string in unique(permutations(seed_vector))]
end;

## **1次元鎖上の Fermi-Hubbard 模型**

In [231]:
function fermi_hubbard_model(num_sites::Integer, num_particles::Integer, t::Number, onsite_interaction::Number, boundary_condition="open")
    num_modes = 2*num_sites
    indices = get_index_from_n(num_particles, num_modes) .+ 1
    dim = 2^num_modes
    hop_ham = spzeros(dim, dim);
    for i in 1:num_sites-1
        hop_ham += t*(cdg(num_modes, 2*i-1)*cop(num_modes, 2*i+1) + cdg(num_modes, 2*i)*cop(num_modes, 2*i+2))
        hop_ham += conj(t)*(cdg(num_modes, 2*i+1)*cop(num_modes, 2*i-1)+ cdg(num_modes, 2*i+2)*cop(num_modes, 2*i))
        hop_ham += onsite_interaction*nop(num_modes, 2*i-1)*nop(num_modes, 2*i)
    end
    hop_ham += onsite_interaction*nop(num_modes, 2*num_sites-1)*nop(num_modes, 2*num_sites)

    if boundary_condition == "periodic"
        hop_ham += t*(cdg(num_modes, 2*num_sites-1)*cop(num_modes, 1) + cdg(num_modes, 2*num_sites)*cop(num_modes, 2))
        hop_ham += conj(t)*(cdg(num_modes, 1)*cop(num_modes, 2*num_sites-1) + cdg(num_modes, 2)*cop(num_modes, 2*num_sites))
    end
    return hop_ham[indices, indices]
end;

In [237]:
fhm_ham = fermi_hubbard_model(5, 2, -1, 5)
eigen(Matrix(fhm_ham))

Eigen{Float64, Float64, Matrix{Float64}, Vector{Float64}}
values:
45-element Vector{Float64}:
 12.04559486222365
 12.267949192431118
 12.267949192431125
 12.267949192431129
 12.97329688278526
 13.267949192431118
 13.267949192431125
 13.267949192431125
 13.612205304744585
 13.999999999999998
 14.0
 14.000000000000005
 14.01356642560259
  ⋮
 16.732050807568864
 16.732050807568864
 16.732050807568882
 17.462745391944164
 17.73205080756888
 17.732050807568882
 17.732050807568886
 19.999999999999993
 20.148974580307225
 20.51273596120879
 20.934259175768407
 21.271135682312092
vectors:
45×45 Matrix{Float64}:
 -0.0274002     5.55112e-17  -1.73472e-16  …  -0.339492      0.16916
 -4.49462e-17   0.0411866     0.0063478        3.20412e-19   2.53831e-20
  0.108976      0.0554085    -0.0457315       -0.158587      0.107513
  1.32789e-16  -0.112524     -0.0173425        6.61755e-19   1.59181e-19
 -0.202897     -0.151379      0.124941        -0.0450878     0.0492362
 -2.01999e-16   0.15371       0.0